In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# # choose a local (colab) directory to store the data.
# local_download_path = ''
# try:
#     os.makedirs(local_download_path)
# except: pass

# # 2. Auto-iterate using the query syntax
# #    https://developers.google.com/drive/v2/web/search-parameters
# file_list = drive.ListFile(
#     {'q': "'1m4t_k5N7-W3saafdfddfdWCqfc1D0xHmc20r' in parents"}).GetList()  #use your own folder ID here

# for f in file_list:
#     # 3. Create & download by id.
#     print('title: %s, id: %s' % (f['title'], f['id']))
#     fname = f['title']
#     print('downloading to {}'.format(fname))
#     f_ = drive.CreateFile({'id': f['id']})
#     f_.GetContentFile(fname)

In [ ]:
import os
relative_path = './data/libri_small/'
if not os.path.exists(relative_path):
  os.mkdir(relative_path)
elif os.path.isfile(f'{relative_path}dataset.arrow'):
  print('dataset already exists on machine')

In [ ]:
hostname=os.popen('hostname').read()[:-1]
hostname

In [ ]:
def assert_type_of_machine():
  try:
    import google.colab
    on_colab = True
  except:
    on_colab = False
    
  if os.popen('hostname').read()[:-1] == 'DESKTOP-3EBED7S':
    on_pc = True
  else:
    on_pc = False
    
  if not on_colab and not on_pc:
    on_server = True
  else:
    on_server = False
    
    
  return on_colab, on_pc, on_server

In [ ]:
def assert_presence_of_dataset_on_machine(dataset='librispeech'):
  """
  Returns True if the dataset is present on the machine or accesible from 
  within Colab. Also returns the relative path to the dataset for further
  use in load_from_disk() given that the dataset is on the machine.
  """
  on_colab, on_pc, on_server = assert_type_of_machine()

  preprocessed_data_available_on_machine = False
  
  if on_colab:
    data_path = '/content/drive/MyDrive/Projects/Cross-Modal Speech Segment Retrieval/cross-modal-speech-segment-retrieval/data/{dataset}/'
    if os.path.isdir(data_path):
      preprocessed_data_available_on_machine = True
  elif on_pc or on_server:
    data_path = f'./data/{dataset}/'
    if os.path.isdir(data_path):
      preprocessed_data_available_on_machine = True

  return preprocessed_data_available_on_machine, data_path

In [ ]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

def assert_presence_of_dataset_on_gdrive(dataset='librispeech'):
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('drive', 'v3', credentials=creds)

        # Call the Drive v3 API
        results = service.files().list(
            pageSize=10, fields="nextPageToken, files(id, name)").execute()
        items = results.get('files', [])

        if not items:
            print('No files found.')
            return
        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')